In [38]:
library(dplyr)
library(tidyr)
library(stringr)
library(geohash)

In [39]:
library(downloader)
# Blight Violations
dbvurl <- "https://d18ky98rnyall9.cloudfront.net/_97bd1c1e5df9537bb13398c9898deed7_detroit-blight-violations.csv?Expires=1484524800&Signature=hlHM02mABfvFUvErGgrV3u31Nr8GO6cwNPX1Z0wzrdY5o8q7faWEsmeoOcGE5p43iH3-WEtliSCduHMa~nrVcxeZpM-vvNhNXIU3D8m~0yBqrjbCpTeVdaC0IA-RgKNN1NfgMHoeSjSItNmjXcVfFq3U5X~xbNuv0DI4No8Qe80_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DBVFileName <- "detroit-blight-violations.csv"
if (!file.exists(DBVFileName)) 
    download(dbvurl, DBVFileName)
detBlightViol <- read.csv("detroit-blight-violations.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )   
    
# Detroit Demolition Permits 
ddpurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-demolition-permits.tsv?Expires=1487116800&Signature=A-ZtSkr8DZMpetuP4NNGPpz327wizHe9iw2SUTvlU5-eOJ1eTgnIflqgmfnaJJ-ultJMdawSbRD-tp5vtc45CnB1E9nMGwO-mFrca3z8bz3HKL09YgJEU2bM571hYeULyucNTcjejsQMmfZno8N2Qs9AwDigjlD1Ixy-pnyk2Ns_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DDPFileName <- "detroit-demolition-permits.tsv"
if (!file.exists(DDPFileName)) 
    download(ddpurl, DDPFileName)
detDemolitionPermit <- read.delim("detroit-demolition-permits.tsv", header=TRUE, allowEscapes=FALSE, sep="\t", na.strings="", comment.char="")

    
# Detroit 311 calls
d311url <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
D311FileName <- "detroit-311.csv"
if (!file.exists(D311FileName)) 
    download(d311url, D311FileName)
det311 <- read.csv("detroit-311.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )  

# Detroit Crime 
dcrurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DCRFileName <- "detroit-crime.csv"
if (!file.exists(DCRFileName)) 
    download(dcrurl, DCRFileName)
detCrime <- read.csv("detroit-crime.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )    
       

In [40]:
#dplyr mutate to extract (LAT,LNG) from ViolationAddress
detBlightViol <- detBlightViol %>%
    mutate(latlng = gsub(".*\\n", "", ViolationAddress)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')
#head(detBlightViol)

#dplyr mutate to extract (LAT,LNG) from site_location
detDemolitionPermit <- detDemolitionPermit %>%
    mutate(latlng = gsub(".*\\n", "", site_location)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')



Warning message:
"Too few values at 14 locations: 18, 23, 28, 88, 89, 94, 99, 144, 172, 189, 194, 206, 227, 335"

In [41]:
#dplyr filter to create gh_8 gh_7 and uid colums 
detBlightViol <- detBlightViol %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('blightviol_',row_number()))

detDemolitionPermit <- detDemolitionPermit %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('demolper',row_number()))

det311 <- det311 %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('det311',row_number()))

detCrime <- detCrime %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('detcrim',row_number()))

In [42]:
dbv_gh_8_grp <- detBlightViol %>%
    group_by(gh_8) %>%
    summarize(dbv_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dbv_ngbr_8_ct))

ddp_gh_8_grp <- detDemolitionPermit %>%
    group_by(gh_8) %>%
    summarize(ddp_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(ddp_ngbr_8_ct))

d311_gh_8_grp <- det311 %>%
    group_by(gh_8) %>%
    summarize(d311_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(d311_ngbr_8_ct))

dcr_gh_8_grp <- detCrime %>%
    group_by(gh_8) %>%
    summarize(dcr_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dcr_ngbr_8_ct))


In [43]:
dbv_gh_7_grp <- detBlightViol %>%
    group_by(gh_7) %>%
    summarize(dbv_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dbv_ngbr_7_ct))

ddp_gh_7_grp <- detDemolitionPermit %>%
    group_by(gh_7) %>%
    summarize(ddp_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(ddp_ngbr_7_ct))

d311_gh_7_grp <- det311 %>%
    group_by(gh_7) %>%
    summarize(d311_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(d311_ngbr_7_ct))

dcr_gh_7_grp <- detCrime %>%
    group_by(gh_7) %>%
    summarize(dcr_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dcr_ngbr_7_ct))


In [44]:
detBlightViolV <- left_join(detBlightViol, dbv_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD <- left_join(detBlightViolV, ddp_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD3 <- left_join(detBlightViolVD, d311_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD3C <- left_join(detBlightViolVD3, dcr_gh_8_grp, by = c("gh_8" = "gh_8"))

In [47]:
detBlightViolVD3C[is.na(detBlightViolVD3C)] <- 0
keep <- c("LAT","LNG","gh_8", "gh_7", "dbv_ngbr_8_ct", "ddp_ngbr_8_ct", "d311_ngbr_8_ct", "dcr_ngbr_8_ct",  "ViolationCategory", "CleanUpCost", "JudgmentAmt", "PaymentStatus", "FineAmt", "AdminFee", "LateFee", "AgencyName")
detBlightViolVD3C[keep]
#head(detBlightViolVD3C)

LAT,LNG,gh_8,gh_7,dbv_ngbr_8_ct,ddp_ngbr_8_ct,d311_ngbr_8_ct,dcr_ngbr_8_ct,ViolationCategory,CleanUpCost,JudgmentAmt,PaymentStatus,FineAmt,AdminFee,LateFee,AgencyName
42.36318237000006,-83.09167672099994,dpsbvpyr,dpsbvpy,12,0,0,0,0,$0.00,$1680.00,PAID IN FULL,$1500.00,$20.00,$150.00,Department of Public Works
42.429390762000025,-83.22039357799997,dpsc6k0c,dpsc6k0,5,0,0,0,0,$0.00,$140.00,NO PAYMENT APPLIED,$100.00,$20.00,$10.00,Department of Public Works
42.428707459000066,-83.22754809599996,dpsc65gt,dpsc65g,3,0,0,0,0,$0.00,$140.00,PAID IN FULL,$100.00,$20.00,$10.00,Department of Public Works
42.44169828400004,-83.14501821599998,dpsc7yr8,dpsc7yr,8,0,0,0,0,$0.00,$140.00,NO PAYMENT APPLIED,$100.00,$20.00,$10.00,Department of Public Works
42.42031769500005,-83.14532817999998,dpsc7frk,dpsc7fr,1,0,0,0,0,$0.00,$140.00,PAID IN FULL,$100.00,$20.00,$10.00,Department of Public Works
42.41788868500004,-83.15363728399996,dpsc7ccq,dpsc7cc,4,0,0,0,0,$0.00,$140.00,NO PAYMENT APPLIED,$100.00,$20.00,$10.00,Department of Public Works
42.41788868500004,-83.15363728399996,dpsc7ccq,dpsc7cc,4,0,0,0,0,$0.00,$140.00,NO PAYMENT APPLIED,$100.00,$20.00,$10.00,Department of Public Works
42.42089943000008,-83.15377362999999,dpsc7f3r,dpsc7f3,3,0,0,0,0,$0.00,$140.00,NO PAYMENT APPLIED,$100.00,$20.00,$10.00,Department of Public Works
42.33168113800008,-83.04799603699996,dpsby4nr,dpsby4n,21114,548,0,0,0,$0.00,$140.00,PARTIAL PAYMENT MADE,$100.00,$20.00,$10.00,Department of Public Works
42.42762085700008,-83.14451351599996,dpsck58n,dpsck58,4,0,0,0,0,$0.00,$140.00,PAID IN FULL,$100.00,$20.00,$10.00,Department of Public Works


In [46]:
library(sqldf)
sqldf("select distinct PaymentStatus from 'detBlightViolVD3C' where ddp_ngbr_8_ct < 5 and ddp_ngbr_8_ct > 2 order by ddp_ngbr_8_ct desc")

PaymentStatus
PAID IN FULL
NO PAYMENT ON RECORD
NO PAYMENT APPLIED
PARTIAL PAYMENT MADE
